### select particular exptypes from full dataset, remove rows and columns which sum is 0
Select rows which: <br>
version == 4.1;  and <br>
row_sum  > 0; and<br>
experiment type == metagenomic or assembly<br>
<br>
old dataset has 74278 rows and new dataset has 36724 rows

In [1]:
import csv
import pandas as pd
import os

In [2]:
data_dir = '/global/cfs/cdirs/kbase/KE-Catboost/ziming/GO/data'
## input file name
input_file = os.path.join(data_dir, '/go_aggregated_v2', 'go_aggregated_with_version_labels_rm_duplicates.tsv')
input_file_pickle = os.path.join(data_dir, 'go_aggregated_v2', 'go_aggregated_with_version_labels_rm_duplicates.pkl')
col_stats_file = os.path.join(data_dir, 'go_aggregated_v2', 'col_stats.tsv')
## output file name
output_file = os.path.join(data_dir, 'go_aggregated_4.1', 'go_aggregated_4.1.tsv')
output_file_pickle = os.path.join(data_dir, 'go_aggregated_4.1', 'go_aggregated_4.1.pkl')

### load original dataset

In [4]:
%%time
if os.path.exists(input_file_pickle):
    ## this step takes 3.59 s
    df = pd.read_pickle(input_file_pickle)
else:
    ## this step takes 5min 38s
    with open(input_file, 'r') as f:
        df = pd.read_csv(f, sep="\t")
        df.to_pickle(input_file_pickle)
df

CPU times: user 647 ms, sys: 2.83 s, total: 3.47 s
Wall time: 3.65 s


,id,study_id,sample_id,biome,exptype,version,GO:0043130,GO:0055074,GO:0055117,GO:0046933,...,GO:0019357,GO:0006527,GO:0004114,GO:0046423,GO:0034194,GO:0032183,GO:0007618,GO:0030097,GO:0004520,GO:0033739
0,SRR3650980,MGYS00002076,SRS1493755,root:Host-associated:Mammals:Respiratory syste...,unknown,4.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,SRR3650981,MGYS00002076,SRS1493754,root:Host-associated:Mammals:Respiratory syste...,unknown,4.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,SRR3650982,MGYS00002076,SRS1493753,root:Host-associated:Mammals:Respiratory syste...,unknown,4.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,SRR3650983,MGYS00002076,SRS1493756,root:Host-associated:Mammals:Respiratory syste...,unknown,4.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,SRR3650984,MGYS00002076,SRS1493757,root:Host-associated:Mammals:Respiratory syste...,unknown,4.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74273,ERZ1694525,MGYS00002035,ERS1030532,root:Environmental:Aquatic:Freshwater:Ice:Glacier,assembly,5.0,0,0,0,151,...,0,36,2,0,6,0,0,0,51,14
74274,ERZ1694526,MGYS00002035,ERS1030533,root:Environmental:Aquatic:Freshwater:Ice:Glacier,assembly,5.0,0,0,0,97,...,0,19,0,0,6,0,0,0,19,10
74275,ERZ1694527,MGYS00002035,ERS1030534,root:Environmental:Aquatic:Freshwater:Ice:Glacier,assembly,5.0,1,0,0,168,...,0,48,0,0,11,0,0,0,52,24
74276,ERZ1694528,MGYS00002035,ERS1030537,root:Environmental:Aquatic:Freshwater:Ice:Glacier,assembly,5.0,1,0,0,166,...,0,53,0,0,13,0,0,0,46,25


In [5]:
## convert version from float number to string
df['version']= df['version'].astype(str)

In [7]:
## Unique biomes
len(set(df['biome']))

205

## select and romove rows

In [5]:
## add a new column "sum" to original dataset
df["sum"] = df.sum(axis=1)

In [6]:
## select version and exptype
df_select = df[(df['version']=='4.1') & ((df['exptype']=='metagenomic') | (df['exptype']=='assembly')) & (df['sum']!=0)].iloc[:, 0:-1]
df_select

,id,study_id,sample_id,biome,exptype,version,GO:0043130,GO:0055074,GO:0055117,GO:0046933,...,GO:0019357,GO:0006527,GO:0004114,GO:0046423,GO:0034194,GO:0032183,GO:0007618,GO:0030097,GO:0004520,GO:0033739
19,ERZ650344,MGYS00003358,SRS3210273,root:Environmental:Aquatic:Marine:Brackish,assembly,4.1,3,0,0,342,...,2,79,90,0,20,0,0,0,87,38
27,ERZ747221,MGYS00004737,SRS1791943,root:Engineered:Biogas plant,assembly,4.1,49,0,0,450,...,2,110,0,0,15,0,0,0,204,34
29,ERZ761615,MGYS00003390,SRS1984850,root:Engineered:Bioreactor,assembly,4.1,2,0,0,270,...,33,102,2,0,43,0,0,0,93,49
30,ERR2193276,MGYS00005065,ERS2001073,root:Host-associated:Mammals,metagenomic,4.1,0,0,0,26,...,0,12,1,0,15,0,0,0,8,5
31,ERR2193277,MGYS00005065,ERS2001074,root:Host-associated:Mammals,metagenomic,4.1,0,0,0,31,...,0,13,1,0,20,0,0,0,10,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73785,ERZ807429,MGYS00005443,ERS1960449,root:Host-associated:Human:Digestive system:Oral,assembly,4.1,0,0,0,3,...,0,0,0,0,0,0,0,0,0,1
74255,ERZ795020,MGYS00004901,SRS785549,root:Engineered:Wastewater,assembly,4.1,0,0,0,115,...,1,10,0,0,1,0,0,0,41,8
74256,ERZ505245,MGYS00003223,SRS1589217,root:Environmental:Aquatic:Marine,assembly,4.1,1,0,0,336,...,3,54,1,0,7,0,0,0,88,56
74261,ERZ505291,MGYS00003237,SRS1589396,root:Environmental:Aquatic:Marine,assembly,4.1,2,0,0,299,...,2,58,0,0,11,0,0,0,74,53


### load col stats file

In [7]:
## remove columns which sum is zero
## load row_stats.tsv
col_stats = pd.read_csv(col_stats_file, sep="\t")
col_stats

,GO,sum
0,GO:0043130,323583
1,GO:0055074,607
2,GO:0055117,2102
3,GO:0046933,103230728
4,GO:0006302,718920
...,...,...
4414,GO:0032183,9524
4415,GO:0007618,1706
4416,GO:0030097,4142
4417,GO:0004520,23156824


In [9]:
zero_sum_cols = col_stats.loc[col_stats['sum']==0]
zero_sum_cols

,GO,sum
72,GO:0005958,0
327,GO:0050105,0
553,GO:0008766,0
691,GO:0006208,0
1089,GO:0045005,0
1125,GO:0015300,0
1266,GO:0015698,0
1393,GO:0009033,0
1923,GO:0006798,0
2007,GO:0004075,0


In [13]:
zero_sum_cols_names = list(zero_sum_cols['GO'].values)
zero_sum_cols_names

['GO:0005958',
 'GO:0050105',
 'GO:0008766',
 'GO:0006208',
 'GO:0045005',
 'GO:0015300',
 'GO:0015698',
 'GO:0009033',
 'GO:0006798',
 'GO:0004075',
 'GO:0003912',
 'GO:0006732',
 'GO:0016725',
 'GO:0016899',
 'GO:0044255',
 'GO:0006578',
 'GO:0016413']

### remove columns whose sum is zero

In [14]:
df_final = df_select.drop(columns=zero_sum_cols_names)

In [17]:
df_final.to_csv(output_file, sep = '\t')
df_final.to_pickle(output_file_pickle)